In [ ]:
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_cbc

In [2]:
CBC = '../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx'

In [ ]:
# Load the data
xls = pd.ExcelFile(CBC)
for sheet_name in xls.sheet_names:
    print(sheet_name)

In [4]:
df_calificaciones = pd.read_excel(CBC, sheet_name='Calificaciones')
df_carreras = pd.read_excel(CBC, sheet_name='Materias Grilla Carreras')

Correcciones que notamos en el eda inicial

In [5]:
df_carreras = helpers_cbc.normalize_column_values(df_carreras, ['Carrera', 'Materia'])
df_calificaciones = helpers_cbc.normalize_column_values(df_calificaciones, ['Carrera', 'Dirección', 'Localidad', 'dominio email', 'Materia', 'Nota', 'UBA XXI', 'Es materia FCEN?'])

Analizamos la relación entre ambas tablas.

Vimos que ambas tablas tienen la misma cantidad de carreras. Verifiquemos que sean las mismas.

In [ ]:
carreras_no_en_grilla = df_calificaciones[~df_calificaciones['Carrera'].isin(df_carreras['Carrera'])]
print(carreras_no_en_grilla)

In [ ]:
carreras_no_en_grilla = df_carreras[~df_carreras['Carrera'].isin(df_calificaciones['Carrera'])]
print(carreras_no_en_grilla)

Se ve que coinciden ambos conjuntos.

En el análisis previo vimos que df_calificaciones tiene 32 materias distintas, mientras que df_carreras tiene tan solo 10.

In [ ]:
materias_no_en_calificaciones = df_carreras[~df_carreras['Materia'].isin(df_calificaciones['Materia'])]
print(materias_no_en_calificaciones)

No hay materias de df_carreras que no aparezcan en df_calificaciones pero no se cumple la inversa.

In [9]:
# Verificar si hay materias que no están en la grilla de carreras
materias_no_en_grilla = df_calificaciones[~df_calificaciones['Materia'].isin(df_carreras['Materia'])]

In [ ]:
df_calificaciones['Materia'].unique()

In [ ]:
materias_no_en_grilla['Materia'].unique()

In [ ]:
df_carreras['Materia'].unique()

In [ ]:
print(materias_no_en_grilla.groupby('Carrera')['Materia'].value_counts().to_markdown())

Vemos que todas estas materias tienen la aclaración de que no son de la FCEN.

In [ ]:
print(materias_no_en_grilla.groupby('Materia')['Es materia FCEN?'].value_counts().to_markdown())

En particular para ALGEBRA A, ANALISIS MATEMATICO y ANALISIS MATEMATICO I, quiero ver si a las personas que cursaron dichas materias se les dio una equivalencia.

In [ ]:
print(df_calificaciones[df_calificaciones['Nota'] == 'AP']['Materia'].value_counts().to_markdown())

Todas las materias tienen un único código.

In [ ]:
print(df_carreras.groupby(['Materia', 'Cód.Mat.']).value_counts().to_markdown())

## Agrupando por DNI

Todas las personas tienen asignada una sola carrera.

In [17]:
cantidad_de_carreras= df_calificaciones.groupby('Dni')['Carrera'].nunique()

In [ ]:
(cantidad_de_carreras>1).sum()

Veamos cuantas materias se suelen inscribir.

In [19]:
cantidad_de_materias= df_calificaciones.groupby('Dni')['Materia'].nunique()

In [ ]:
# Definir los bins como bordes enteros desde el mínimo hasta el máximo + 1
bins = range(cantidad_de_materias.min(), cantidad_de_materias.max() + 2)

plt.figure(figsize=(10, 6))
plt.hist(cantidad_de_materias, bins=bins, edgecolor='black', align='left')
plt.title('Distribución de cantidad de materias inscriptas por estudiante')
plt.xlabel('Cantidad de materias')
plt.ylabel('Cantidad de estudiantes')
plt.xticks(range(cantidad_de_materias.min(), cantidad_de_materias.max() + 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [21]:
df_dni_carrera = df_calificaciones[['Dni', 'Carrera']].drop_duplicates()

In [ ]:
# Estilo general
sns.set(style="whitegrid")

# Calcular proporciones
carrera_proporciones = df_dni_carrera['Carrera'].value_counts(normalize=True).sort_values()

# Crear el gráfico
plt.figure(figsize=(10, 6))
ax = carrera_proporciones.plot(kind='barh', color=sns.color_palette("crest", len(carrera_proporciones)))

# Agregar etiquetas con porcentajes
for i, (index, value) in enumerate(carrera_proporciones.items()):
    plt.text(value + 0.002, i, f"{value:.1%}", va='center')

# Títulos y etiquetas
plt.title('Distribución proporcional de estudiantes por carrera', fontsize=14, weight='bold')
plt.xlabel('Proporción')
plt.ylabel('Carrera')
plt.xlim(0, carrera_proporciones.max() + 0.05)

plt.tight_layout()
plt.show()

In [23]:
df_mail = df_calificaciones[['Dni', 'dominio email']].drop_duplicates()
# Agrupar dominios: si es gmail.com, se queda; si no, se reemplaza por "OTROS"
df_mail['dominio agrupado'] = df_mail['dominio email'].apply(
    lambda x: x if x == 'GMAIL.COM' else 'OTROS'
)

In [ ]:
df_mail['dominio agrupado'].value_counts(normalize=True).plot(kind='bar', color=['#1f77b4', '#ff7f0e'])
plt.title('Distribución de dominios de email')
plt.ylabel('Proporción')
plt.xticks(rotation=0)
plt.show()

In [ ]:
df_calificaciones.columns

In [26]:
df_localidad= df_calificaciones[['Dni', 'Localidad']].drop_duplicates()

In [ ]:
df_localidad.nunique()

In [ ]:
top_10_localidades = df_localidad['Localidad'].value_counts(normalize = True).head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_localidades.values, y=top_10_localidades.index, palette="viridis")
plt.title('Top 10 localidades con más estudiantes (DNI únicos)', fontsize=14, weight='bold')
plt.xlabel('Cantidad de estudiantes')
plt.ylabel('Localidad')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Reemplazar NaN por una categoría explícita
df_localidad['Localidad'] = df_localidad['Localidad'].fillna('Sin dato')

# Contar localidades (ahora incluye 'Sin dato') y tomar el top 10
top_localidades = df_localidad['Localidad'].value_counts(normalize = True).head(10)

# Visualización
plt.figure(figsize=(10, 6))
sns.barplot(x=top_localidades.values, y=top_localidades.index, palette="mako")
plt.title('Top 10 localidades con más estudiantes (incluye "Sin dato")', fontsize=14, weight='bold')
plt.xlabel('Cantidad de estudiantes')
plt.ylabel('Localidad')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
df_anio = df_calificaciones[['Dni', 'Fecha']]
df_anio['mes'] = df_calificaciones['Fecha'].dt.month
df_anio['anio'] = df_calificaciones['Fecha'].dt.year

In [ ]:
df_anio['anio'].value_counts().sort_index().plot(kind='bar', color='skyblue')

In [ ]:
df_anio['anio'].value_counts()

In [ ]:
df_anio.groupby('anio')['mes'].value_counts().sort_index().plot(kind='bar', color='skyblue')

In [34]:
df_anio_examenes_rendidos = df_calificaciones[(df_calificaciones['Nota'] != 'A') & (df_calificaciones['Nota'] != 'NR')][['Dni', 'Fecha']]
df_anio_examenes_rendidos['mes'] = df_anio_examenes_rendidos['Fecha'].dt.month
df_anio_examenes_rendidos['anio'] = df_anio_examenes_rendidos['Fecha'].dt.year

In [ ]:
df_anio_examenes_rendidos['anio'].value_counts().sort_index().plot(kind='bar', color='skyblue')

In [ ]:
df_anio_examenes_rendidos.groupby('anio')['mes'].value_counts().sort_index().plot(kind='bar', color='skyblue')